In [1]:
from netqasm.sdk.classical_communication.socket import Socket
from netqasm.sdk.connection import BaseNetQASMConnection
from netqasm.sdk.epr_socket import EPRSocket
from netqasm.sdk.qubit import Qubit
from netqasm.sdk.toolbox.state_prep import set_qubit_state


from squidasm.sim.stack.program import Program, ProgramContext , ProgramMeta


import random
import numpy as np



In [2]:
class AliceProgram(Program):
    PEER_NAME = "Bob"

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name = "tutorial_program",
            csockets = [self.PEER_NAME],
            epr_sockets = [self.PEER_NAME],
            max_qubits = 4,
        )

    def run(self, context: ProgramContext):
        # get classical socket to peer
        csocket = context.csockets[self.PEER_NAME]
        # get EPR socket to peer
        epr_socket = context.epr_sockets[self.PEER_NAME]
        #get connection to quantum network processing unit
        connection = context.connection

        # send a string message via a classical channel
        message = "0.25"
        csocket.send(message)
       # print(f"Alice sends message: {message}")

        # Register a request to create an EPR pair, then apply hadamard gate on the epr qubit and measure
        epr_qubit = epr_socket.create_keep()[0]
        epr_qubit.H()
        result = epr_qubit.measure()
        yield from connection.flush()
        #print(f"Alice measures local EPR qubit:{result}")

        #Qubits on a local node can be obtained, but require the connection to be initialized
        #Create a cluster state


        local_qubit0 = Qubit(connection)
        local_qubit1 = Qubit(connection)
        local_qubit2 = Qubit(connection)
        local_qubit3 = Qubit(connection)

        list_phi = [np.pi/4 , 2*np.pi/4 , 3*np.pi/4 , 4*np.pi/4 , 5*np.pi/4 , 6*np.pi/4 , 7*np.pi/4 ]
        theta_1 = random.choice(list_phi)
        theta_2 = random.choice(list_phi)
        
        #INITIALISATION DES QUBITS DE L'ETAT DE CLUSTER
        set_qubit_state(local_qubit0 , phi = 0.0 , theta = np.pi/2)
        set_qubit_state(local_qubit1 , phi = theta_1 , theta = np.pi/2)
        set_qubit_state(local_qubit2 , phi = theta_2 , theta = np.pi/2)
        set_qubit_state(local_qubit3 , phi = 0.0 , theta = np.pi/2)
        
        #Maintenant je mets la connections entre les qubits
        local_qubit0.cphase(local_qubit1)
        local_qubit1.cphase(local_qubit2)
        local_qubit2.cphase(local_qubit3)
       

        #Application of U opérator for make local complémentation on qubit 2
        # racine carré de  Z sur le qubit 1
        local_qubit1.rot_Z(1 , 1)

        # racine carrée de X sur le qubit 2
        local_qubit2.H()
        local_qubit2.rot_Z(1 , 1)
        local_qubit2.H()

        # #racine carrée de Z sur le qubit 3
        local_qubit3.rot_Z(1 , 1)
        
        # # mesure pour marquer l'état rechercher. on marque l'état |10>
        phi_1 = np.pi/2
        phi_2 = 0
        
        r_1 = random.choice([0 , 1])
        r_2 = random.choice([0 , 1])
        delta_1 = theta_1 + np.pi*r_1 + phi_1
        delta_2 = theta_2 + np.pi*r_2 + phi_2

        
        # qubit 1 angle delta_1
        local_qubit1.H()
        local_qubit1.rot_Z(angle = delta_1)
        #qubit 2
        local_qubit2.H()
        local_qubit2.rot_Z(angle = delta_2)

        r1 = local_qubit1.measure()
        r2 = local_qubit2.measure()


        #trouver l'élément cible: 
        local_qubit0.H()
        local_qubit0.rot_Z(1 , 1)

        local_qubit3.H()
        local_qubit3.rot_Z(1 , 1)

        
        #local_qubit0.H()
        # r0 = local_qubit0.measure(basis_rotations= (0 , 8 , 8))
        # r3 = local_qubit3.measure(basis_rotations= (0 , 8 , 8))
        r0 = local_qubit0.measure()
        r3 = local_qubit3.measure()

        yield from connection.flush()
        
        #print(f"Alice measures local qubits: {r0} , {r1} , {r2} , {r3}")
      
        #return {}
        return {'r0': r0, 'r3': r3}

In [3]:
class BobProgram(Program):
    PEER_NAME = "Alice"

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name = "tutorial_program",
            csockets=[self.PEER_NAME],
            epr_sockets=[self.PEER_NAME],
            max_qubits=4,
        )

    def run(self, context:ProgramContext):
        #get classical socket to peer
        csocket: Socket = context.csockets[self.PEER_NAME]
        #get EPR socket to peer
        epr_socket: EPRSocket = context.epr_sockets[self.PEER_NAME]
        #get connection to quantum network processing unit
        connection: BaseNetQASMConnection = context.connection

        # Bob listens for messages on his classical socket
        message = yield from csocket.recv()
        #print(f"Bob receives message: {message}")

        # Listen for request to create EPR pair, apply a Hadamard gate on the epr qubit and measure
        epr_qubit = epr_socket.recv_keep()[0]
        epr_qubit.H()
        result = epr_qubit.measure()
        yield from connection.flush()
        #print(f"Bob measures local EPR qubit:{result}")
        return {}

